<a href="https://colab.research.google.com/github/dominique-nshimyimana/Karat-GC/blob/master/Copy_of_The_tensorflow_version_magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TensorFlow versions in Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from google.colab import drive
mounted = '/drive'
drive.mount(mounted)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


# NN state Evaluation

In [0]:
import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split


class NeuralDynamics(tf.keras.Model):

    def __init__(self, state_size, action_size):

        super(NeuralDynamics, self).__init__()

        self.state_size = state_size
        self.action_size = action_size

        self.layer1 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.layer2 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        '''
        self.layer_mat1 = tf.keras.layers.Dense(128, activation='relu')
        self.layer_mat2 = tf.keras.layers.Dense((self.state_size*(self.state_size + self.action_size)), kernel_regularizer=tf.keras.regularizers.l1(0.1))
        '''

        self.layer_state1 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.layer_state2 = tf.keras.layers.Dense(self.state_size, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.known_inputs = False
        self.training_step = 0

    @tf.function
    def call(self, inputs):

        state, action = inputs
        if not self.known_inputs:
            #model._set_inputs((tf.keras.Input(shape=state.get_shape()), tf.keras.Input(shape=action.get_shape())))
            self.known_inputs = True

        x = tf.concat((state, action), axis=1)
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)

        '''
        mat = self.layer_mat1(x)
        mat = self.dropout(mat)
        mat = self.layer_mat2(mat)
        mat = tf.reshape(mat, shape=(tf.shape(mat)[0], self.state_size, self.state_size + self.action_size), name="Mat")
        '''
        x = self.layer_state1(x)
        x = self.dropout(x)
        state = self.layer_state2(x)
        return state#, mat

    @tf.function
    def linearize(self, state, action):
        inputs = (tf.convert_to_tensor(state), tf.convert_to_tensor(action))
        with tf.GradientTape() as tape:
            tape.watch(inputs)
            state = self.call(inputs)
        state_grad, action_grad = tape.gradient(state, inputs)
        return state_grad


decay=tf.keras.optimizers.schedules.ExponentialDecay(0.0004, 20000, 0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=decay)
#optimizer = tf.keras.optimizers.RMSprop(learning_rate=decay)
model = NeuralDynamics(6, 2)
training_counter = 0


def get_losses(s, a, s_):

    #states_pred, mats_pred = model((s, a))
    states_pred = model((s, a))
    #predictions_mat = (mats_pred @ tf.concat((s, a), axis=1)[..., None])[..., 0]
    #loss_mat = tf.reduce_mean(tf.keras.losses.mean_squared_error(s_, predictions_mat))
    loss_state = tf.reduce_mean(tf.keras.losses.mean_squared_error(s_, states_pred))
    loss = loss_state #+ loss_mat

    #return loss, loss_mat, loss_state
    return loss, loss_state


def train_step(s, a, s_):
    global training_counter
    with tf.GradientTape() as tape:
        #loss, loss_mat, loss_state = get_losses(s, a, s_)
        loss,  loss_state = get_losses(s, a, s_)
        if training_counter % 500 == 0:
            print("The training loss for state is: {}".format(loss_state))
            #print("The training loss for mat is: {}".format(loss_mat))
            pass

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    training_counter += 1
    #if loss < 0.5:
    #    model.save(f'/home/kuba/NeuralModel/{loss:.2f}_{training_counter}/')

def evaluate(s, a, s_):

    #loss, loss_mat, loss_state = get_losses(s, a, s_)
    loss, loss_state = get_losses(s, a, s_)
    #return loss, loss_mat, loss_state
    return loss, loss_state


EPOCHS = 50
data_dir = "/drive/My Drive/neuronyte_logging"
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) and ".directory" not in f]

cutoff_beginning = 300
cutoff_end = 1000

xs_states = []
xs_actions = []
ys = []

for f in data_files:
    print(f)
    data = np.loadtxt(f, delimiter=', ', skiprows=1, dtype=np.float32)[cutoff_beginning:-cutoff_end, :]
    x_states = [data[i, 1:-3] for i in range(len(data))]
    x_actions = [data[i, -3:] for i in range(len(data))]
    y = x_states[1:]
    x_states, x_actions = x_states[:-1], x_actions[:-1]
    xs_states += x_states
    xs_actions += x_actions
    ys += y

xs_states = np.vstack(xs_states)
xs_actions = np.vstack(xs_actions)
ys = np.vstack(ys)
brake_mask = xs_actions[:, 0] < xs_actions[:, 1]
xs_actions[brake_mask, 0] = -xs_actions[brake_mask, 1]
xs_actions = xs_actions[:, [0, 2]]

xs_states_shape_before = xs_states.shape[0]
xs_std_before = np.std(xs_states, axis=0)
ys_std_before = np.std(ys, axis=0)

inliers = ~np.any(abs(xs_states - np.mean(xs_states, axis=0)) >= 3*np.std(xs_states, axis=0), axis=1)
xs_states = xs_states[inliers]
xs_actions = xs_actions[inliers]
ys = ys[inliers]

inliers = ~np.any(abs(ys - np.mean(ys, axis=0)) >= 3*np.std(ys, axis=0), axis=1)
xs_states = xs_states[inliers]
xs_actions = xs_actions[inliers]
ys = ys[inliers]

print("Mean:")
print(np.mean(xs_states, axis=0))
print(np.max(xs_states, axis=0))
#xs_states -= np.mean(xs_states, axis=0)
#xs_states /= np.std(xs_states, axis=0)

print("Mean after normalization:")
print(np.mean(xs_states, axis=0))
print(np.max(xs_states, axis=0))

#ys -= np.mean(ys, axis=0)
#ys /= np.std(ys, axis=0)

xs_states_shape_after = xs_states.shape[0]
xs_std_after = np.std(xs_states, axis=0)
ys_std_after = np.std(ys, axis=0)

print(f"Dropped {xs_states_shape_before-xs_states_shape_after} outliers, size before {xs_states_shape_before}, size after {xs_states_shape_after}")
print(xs_std_before)
print(xs_std_after)
print(ys_std_before)
print(ys_std_after)


print(xs_states.shape, xs_actions.shape, ys.shape)

x_states_train, x_states_test, x_actions_train, x_actions_test, y_train, y_test = train_test_split(xs_states, xs_actions, ys)
print(x_states_train.shape, x_actions_train.shape, y_train.shape)
print(x_states_test.shape, x_actions_test.shape, y_test.shape)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_states_train, x_actions_train, y_train)).shuffle(500000).batch(128)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_states_test, x_actions_test, y_test)).shuffle(500000).batch(16)

for epoch in range(EPOCHS):

    print(f"Epoch {epoch}/{EPOCHS}")
    for x_state, x_action, y in train_ds:
        train_step(x_state, x_action, y)

    total_loss = 0
    total_loss_mat = 0
    total_loss_state = 0
    i = 0
    for x_state, x_action, y in test_ds:
        i += 1
        #loss, loss_mat, loss_state = evaluate(x_state, x_action, y)
        loss, loss_state = evaluate(x_state, x_action, y)
        total_loss += loss
       # total_loss_mat += loss_mat
        total_loss_state += loss_state
        #print(model.linearize(x_state, x_action))
        '''
        if epoch > 10 and loss > 100:
            print("High loss states:")
            for x, a, y_ in zip(x_state, x_action, y):
                loss, loss_mat, loss_state = evaluate(tf.expand_dims(x, axis=0), tf.expand_dims(a, axis=0), tf.expand_dims(y_, axis=0))
                print("----------------------------")
                print(f"Total loss: {loss}, mat loss: {loss_mat}, state_loss: {loss_state}")
                print("State:")
                print(x)
                print("Action:")
                print(a)
                print("Result:")
                print(y_)
        '''

    print('\033[94m' + f"After {epoch} epochs the validation total loss is: {total_loss/i}" + '\033[0m')
    print('\033[94m' + f"After {epoch} epochs the validation mat loss is: {total_loss_mat/i}" + '\033[0m')
    print('\033[94m' + f"After {epoch} epochs the validation state loss is: {total_loss_state/i}" + '\033[0m')


train_ds = tf.data.Dataset.from_tensor_slices(
    (x_states_train, x_actions_train, y_train)).shuffle(500000).batch(32)

for x_states_train, x_actions_train, _ in train_ds:
    print("predicitng")
    model.predict((x_states_train, x_actions_train))
    break
model.save(join(mounted, 'My Drive/NeuralModel/3/') )

/drive/My Drive/neuronyte_logging/NeuroNyte_1585748620.316643
/drive/My Drive/neuronyte_logging/NeuroNyte_1585745385.311904
Mean:
[-1.3224991e+02 -1.1697558e+02  1.3384195e-01  2.7793705e-01
 -1.5106058e-01 -3.9884944e-02]
[10.6461  26.519    3.14158 26.3138  26.2639   9.53066]
Mean after normalization:
[-1.3224991e+02 -1.1697558e+02  1.3384195e-01  2.7793705e-01
 -1.5106058e-01 -3.9884944e-02]
[10.6461  26.519    3.14158 26.3138  26.2639   9.53066]
Dropped 3299 outliers, size before 1053425, size after 1050126
[74.48583   50.47957    1.8485619 11.08395   10.294982  23.197948 ]
[74.54682   50.47944    1.8488507 11.062421  10.309836   1.0919988]
[74.48564   50.47938    1.8485597 11.083953  10.294984  23.197948 ]
[74.54663   50.47921    1.8488514 11.062365  10.309894   1.0919839]
(1050126, 6) (1050126, 2) (1050126, 6)
(787594, 6) (787594, 2) (787594, 6)
(262532, 6) (262532, 2) (262532, 6)
Epoch 0/50
The training loss for state is: 7183.607421875
The training loss for state is: 0.09239158

# Visualization of result